# Semantic Kernel Agent Solution

## Prepare the files

In [1]:
import os

file_directory = "../Data/products"

# Get our files in the file directory
try:
    filenames = os.listdir(file_directory)
    print(filenames)
except FileNotFoundError:
    print(f"Directory '{file_directory}' not found.")

def get_filepath_for_filename(filename: str) -> str:
    base_directory = file_directory
    return os.path.join(base_directory, filename)



['product_info_18.pdf', 'product_info_2.pdf', 'product_info_7.pdf', 'product_info_1.pdf', 'product_info_6.pdf', 'product_info_8.pdf', 'product_info_4.pdf', 'product_info_3.pdf', 'product_info_5.pdf', 'product_info_16.pdf', 'product_info_14.pdf', 'product_info_11.pdf', 'product_info_9.pdf', 'product_info_12.pdf', 'product_info_20.pdf', 'product_info_17.pdf', 'product_info_10.pdf', 'product_info_13.pdf', 'product_info_15.pdf', 'product_info_19.pdf']


## Create a Plugin

In [4]:
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from typing import Annotated

# Define a sample plugin for the hiking company
class HikingPlugin:
    """A sample Hiking Plugin for the hiking company."""

    @kernel_function(description="Provides a list of hiking destinations.")
    def get_destinations(self) -> Annotated[str, "Returns the hiking destinations that you can explore."]:
        return """
            United States
            Australia
            France
        """

    @kernel_function(description="Provides the typical hiking budget for a destination")
    def get_hiking_budget(self, 
        destination: Annotated[str, "The name of the hiking destination"]) -> Annotated[str, "Returns the budget for the destination."]:
        if destination == "United States":
            return """
                Budget Hikers: Around $50 per day. This includes camping, cooking your own meals, and using public transportation.
                Mid-Range Hikers: Approximately $150 per day. This covers mid-range accommodations, dining at average restaurants, and some paid attractions.
                Luxury Hikers: About $400 per day. This includes luxury lodges, fine dining, and private transportation.
            """
        elif destination == "Australia":
            return """
                Budget Hikers: Around $40 per day. This includes camping, cooking your own meals, and using public transport.
                Mid-Range Hikers: Approximately $120 per day. This covers mid-range accommodations, dining at average restaurants, and some paid attractions.
                Luxury Hikers: About $350 per day. This includes luxury lodges, fine dining, and private transportation.
            """
        elif destination == "France":
            return """
                Budget Hikers: Around $45 per day. This includes camping, cooking your own meals, and using public transportation.
                Mid-Range Hikers: Approximately $130 per day. This covers mid-range accommodations, dining at average restaurants, and some paid attractions.
                Luxury Hikers: About $380 per day. This includes luxury lodges, fine dining, and private transportation.            
            """
        else:
            return "Destination not found"
    
    @kernel_function(description="Provides the weather for a hiking destination")
    def get_weather(self, 
        destination: Annotated[str, "The name of the hiking destination"]) -> Annotated[str, "Returns the weather for the destination."]:
        if destination == "United States":
            return """
                The United States has a diverse climate. In general:
                - **Winter**: Cold in the north, mild in the south. Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.
                - **Summer**: Hot and humid in the south, dry in the west. Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).
            """
        elif destination == "Australia":
            return """
                Australia experiences varied climates:
                - **Winter**: Mild in the north, cooler in the south. Average temperatures range from 46°F (8°C) in Canberra to 77°F (25°C) in Darwin.
                - **Summer**: Hot and dry in the interior, humid in the north. Average temperatures range from 68°F (20°C) in Hobart to 91°F (33°C) in Darwin.
            """
        elif destination == "France":
            return """
                France has several climate zones:
                - **Winter**: Cold in the north, mild in the south. Average temperatures range from 35.5°F (2°C) in the northeast to 48°F (9°C) in the south.
                - **Summer**: Warm and sunny. Average temperatures range from 63°F (17°C) in the north to 77°F (25°C) along the Mediterranean coast.
            """
        else:
            return "Destination not found"

## Reformat citations with the proper filenames

In [2]:
from semantic_kernel.contents.annotation_content import AnnotationContent

async def reformat_citations(agent, response):
    # Extract the annotations
    annotations = [item for item in response.items if isinstance(item, AnnotationContent)]
    
    # Original response
    paragraph = response.content
    
    # Dictionary to store key-value pairs of text and filename
    text_filename_pairs = {}

    # Iterate over the annotations and extract the relevant information
    for annotation in annotations:
        file_id = annotation.file_id
        text = annotation.quote
        # Retrieve the filename from the file_id
        cited_file = await agent.client.files.retrieve(file_id)
        filename = cited_file.filename

        if text not in text_filename_pairs:
            text_filename_pairs[text] = []
        text_filename_pairs[text].append(filename)

    # Replace the citation texts with their corresponding filenames prefixed with " Source: "
    for text, filenames in text_filename_pairs.items():
        sources = " Source: " + ", ".join(filenames)
        paragraph = paragraph.replace(text, sources)

    return paragraph


## Step 1-2: Create an Agent and Thread

In [5]:
from semantic_kernel.agents.open_ai.azure_assistant_agent import AzureAssistantAgent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.annotation_content import AnnotationContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel

# Step 0: Create a kernel and add the plugin
kernel=Kernel()
kernel.add_plugin(HikingPlugin(), plugin_name="HikingPlugin")

# Step 1: Create an assistant agent
agent = await AzureAssistantAgent.create(
        kernel=kernel,
        service_id="agent",
        name="SK_OpenAI_Assistant_Agent_File_Search_with_Functions",
        instructions="""
            You are provided a document store and tools to search for information.
            The document store contains Hiking products of Contoso company.
            Always analyze the document store and tools to provide an answer to the user's question.
            Never rely on your knowledge of information not included in the document store and tools.
            Always format response using markdown.
            """,
        enable_file_search=True,
        vector_store_filenames=[get_filepath_for_filename(filename) for filename in filenames],
    )

# Step 2: Create a thread
thread_id = await agent.create_thread()

## Step 3-6: Helper Function
3. Add a message to the thread
4. Run the Assistant
5. Display the Assistant's Response

In [6]:
async def run_assistant(user_question):
    # STEP 3: Add a user question to the thread
    await agent.add_chat_message(
            thread_id=thread_id, 
            message=ChatMessageContent(role=AuthorRole.USER, content=user_question)
    )

    # STEP 4: Invoke the agent to get a response
    async for response in agent.invoke(thread_id=thread_id):
        annotations = [item for item in response.items if isinstance(item, AnnotationContent)]
        #STEP 5: Print the Assistant response
        if annotations is None:
            print(f"{response.content}", end="", flush=True)
        else:
            print(f"{await reformat_citations(agent,response)}", end="", flush=True)

In [7]:
user_question = "What is the price of the SummitClimber Backpack?"
await run_assistant(user_question)

The price of the SummitClimber Backpack is $120 Source: product_info_9.pdf, product_info_2.pdf.

## Appending Messages to the Thread

In [8]:
user_question = "What countries can I hike?"
await run_assistant(user_question)

You can hike in the following countries:

- United States
- Australia
- France

In [9]:
user_question = "If I go to the US, how much money do I need for a 3 day hike and what would the weather be like?"
await run_assistant(user_question)

### Hiking Budget for a 3-Day Hike in the United States:

- **Budget Hikers**: Around $50 per day. Total for 3 days: $150 (Includes camping, cooking your own meals, and using public transportation).
- **Mid-Range Hikers**: Approximately $150 per day. Total for 3 days: $450 (Covers mid-range accommodations, dining at average restaurants, and some paid attractions).
- **Luxury Hikers**: About $400 per day. Total for 3 days: $1200 (Includes luxury lodges, fine dining, and private transportation).

### Weather in the United States:

The United States has a diverse climate. In general:
- **Winter**: Cold in the north, mild in the south. Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.
- **Summer**: Hot and humid in the south, dry in the west. Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).

In [10]:
user_question = """
I have a total budget of $1000. 
If I will hike in the US for 3 days on a budget, what products can you recommend that I buy with the money left?
"""
await run_assistant(user_question)

Since you have a total budget of $1000 and a 3-day budget hike in the US costs $150, you have $850 left to spend on hiking products. Here are some products you can consider purchasing with this remaining budget:

1. **TrailBlaze Hiking Pants** - $75
   - **Brand**: MountainStyle
   - Lightweight, quick-drying, water-resistant, breathable, with multiple zippered pockets Source: product_info_9.pdf, product_info_17.pdf.

2. **TrekStar Hiking Sandals** - $70
   - **Brand**: TrekReady
   - Durable, lightweight, quick-drying with adjustable straps and cushioned footbed Source: product_info_9.pdf, product_info_17.pdf.

3. **RainGuard Hiking Jacket** - $110
   - **Brand**: MountainStyle
   - Waterproof, breathable, adjustable hood and cuffs, multiple pockets, reflective details Source: product_info_2.pdf, product_info_17.pdf Source: product_info_7.pdf, product_info_17.pdf, product_info_7.pdf, product_info_17.pdf.

4. **TrekReady Hiking Boots** - $140
   - Durable, comfortable fit, excellent tr

## Deleting Files, Thread, Agent

In [11]:
if agent is not None:
    [await agent.delete_file(file_id) for file_id in agent.file_search_file_ids]
    await agent.delete_thread(thread_id)
    await agent.delete()
